# Bi-Metrics
Por Andrea Fuentes Pérez y Víctor Sánchez Auñón
/ 3º de Ingeniería Biomédica

## VALIDACIÓN Y FORMATO DEL MRZ CON PRE-PROCESIONAMIENTO:

In [1]:
from doctr.io import DocumentFile
from doctr.models import ocr_predictor
from PIL import Image, ImageEnhance
import os
from datetime import date

mrz = {
    "linea1": "",
    "linea2": "",
    "linea3": ""
}

model = ocr_predictor(pretrained=True)

def extract_text(ocr_model, image_file_path: str, angulo: float = 0, 
                 algoritmo: Image.Resampling = Image.Resampling.NEAREST, idCorte: int = 0):
    """Extrae el texto de una imagen a partir de un modelo OCR"""
    
    # Abrimos, cropeamos, giramos y aumentamos para mejor lectura
    img = Image.open(image_file_path).convert('L')

    ancho, alto = img.size
    # (inicioX, inicioY, finX, finY)
    corte = [(0, 0, ancho, alto), #0 imagen completa
             (0, alto*0.6, ancho, alto), #1 parte inferior (MRZ)
             (ancho*0.433, alto*0.17, ancho*0.80, alto*0.5), #2 DNI
             (ancho*0.35, alto*0.25, ancho*0.6, alto*0.64), #3 Apellidos
             (ancho*0.35, alto*0.37, ancho*0.6, alto*0.6), #4 Nombre
             (ancho*0.30, alto*0.48, ancho*0.55, alto*0.80), #5 Sexo
             (ancho*0.50, alto*0.48, ancho*0.70, alto*0.60), #6 Nacionalidad
             (ancho*0.75, alto*0.48, ancho*1, alto*0.80), #7 Fecha de nacimiento
             (ancho*0.30, alto*0.57, ancho*0.6, alto*1), #8 Fecha de emisión y número de soporte
             (ancho*0.58, alto*0.57, ancho*1, alto*1), #9 Fecha de expiración
             (0, alto*0.18, ancho*0.3, alto*0.28)] #10 DNI (por si falla)

    img = img.crop(corte[idCorte])
    img = img.rotate(angulo, resample=algoritmo, expand=True)  
    img = ImageEnhance.Contrast(img).enhance(2.0)    
    temp_path = "temp_mrz_process.jpg"
    img.save(temp_path)

    doc = DocumentFile.from_images(temp_path)
    result = ocr_model(doc)
    
    # Limpiar archivo temporal
    if os.path.exists(temp_path):
        os.remove(temp_path)
        
    return result

def show_ocr_result(result):
    """Muestra una imagen de resultado y el texto extraido"""

    # mostrar resultado
    result.show()

    # mostrar texto con mas de 50% de confianza
    for page in result.pages:
        for block in page.blocks:
            for line in block.lines:
                words = []
                for word in line.words:
                    if word.confidence > 0.5:
                        words.append(word.value)
                
                
                print(' '.join(words).upper())

#Primera estraregia: Algoritmo BICUBIC, angulo de 0.5 para la línea 1 y 2, 0.3 para línea 3
def leerMRZ(path):
    text_back1 = extract_text(model, path, angulo=0.5, algoritmo=Image.Resampling.BICUBIC, idCorte=1)
    text_back2 = extract_text(model, path, angulo=0.3, algoritmo=Image.Resampling.BICUBIC, idCorte=1)

    mrz = {
        "linea1": text_back1.pages[0].blocks[0].lines[0].words[0].value,
        "linea2": text_back1.pages[0].blocks[0].lines[1].words[0].value,
        "linea3": text_back2.pages[0].blocks[0].lines[2].words[0].value
    }

    mal = False
    for key, value in mrz.items():
        if len(value) != 30:
            mal = True

    if mal:
        print("\nLectura incorrecta. Probando con el siguiente algoritmo...\n")
        estrategia2MRZ(path)
    else:
        return mrz

#Segunda estrategia: Algoritmo NEAREST, angulo de 0.5 para las tres líneas
def estrategia2MRZ(path):
    text_back = extract_text(model, path, angulo=0.5, algoritmo=Image.Resampling.NEAREST, idCorte=1)

    mrz = {
        "linea1": text_back.pages[0].blocks[0].lines[0].words[0].value,
        "linea2": text_back.pages[0].blocks[0].lines[1].words[0].value,
        "linea3": text_back.pages[0].blocks[0].lines[2].words[0].value
    }

    mal = False
    for key, value in mrz.items():
        print(f"{key}: {value}")
        if len(value) != 30:
            mal = True

    if mal:
        print("\nLectura incorrecta. Probando con el siguiente algoritmo...\n")
        estrategia3MRZ(path)
    else:
        return mrz

#Tercera estrategia: Algoritmo BILINEAL, angulo de 0.5 para las tres líneas
def estrategia3MRZ(path):
    text_back = extract_text(model, path, angulo=0.5, algoritmo=Image.Resampling.BILINEAR, idCorte=1)

    mrz = {
        "linea1": text_back.pages[0].blocks[0].lines[0].words[0].value,
        "linea2": text_back.pages[0].blocks[0].lines[1].words[0].value,
        "linea3": text_back.pages[0].blocks[0].lines[2].words[0].value
    }

    mal = False
    for key, value in mrz.items():
        print(f"{key}: {value}")
        if len(value) != 30:
            mal = True

    if mal:
        print("\nLectura incorrecta, por favor repita la imagen\n")
    else:
        return mrz

c:\Users\vicsa\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
#Obtenemos MRZ de una imagen
mrz = leerMRZ('./dni_fotos/back.jpg')

for key, value in mrz.items():
    print(f"{key}: {value}")

linea1: IDESPCAA000000499999999R<<<<<<
linea2: 8001014F3106028ESP<<<<<<<<<<<1
linea3: ESPANOLA<ESPANOLA<<CARMEN<<<<<


## CÁLCULO Y COMPROBACIÓN DE CHECKSUMS:

In [5]:
for key, value in mrz.items():
    print(f"{key}: {value}")
print("\n")

num_docu = mrz["linea1"][5:14]
fecha_nac = mrz["linea2"][0:6]
#control es L1 sin ID ni pais (5 al final) + L2 sin sexo ni nacionalidad (0 a 7 + 8 a 15 + 18 a 29)
control = mrz["linea1"][5:] + mrz["linea2"][0:7] + mrz["linea2"][8:15] + mrz["linea2"][18:29]
def calculoHash(campo):
    total = 0
    posi = 0 #empieza en 0 -> *7, posi++ 1 -> *3, posi++ 2 -> *1 y posi - 2 
    valoresHash = [7, 3, 1]
    for char in campo:
        total += valorCharHash(char) * valoresHash[posi]
        
        if posi == 2:
            posi = 0
        else:
            posi += 1
    return total % 10 # cheksum = resto de 10 (es el ultimo digito)

def valorCharHash(char):
    if char.isdigit():
        return int(char)
    elif char == "<":
        return 0
    else:
        # A=65, si tiene q valer 10, entonces ord(A) - 55 = 10
        return ord(char) - 55

print("Hash numero documento: ", calculoHash(num_docu)) #4
print("Hash fecha nacimiento: ", calculoHash(fecha_nac)) #8
print("Hash control final: ", calculoHash(control)) #7


linea1: IDESPCAA000000499999999R<<<<<<
linea2: 8001014F3106028ESP<<<<<<<<<<<1
linea3: ESPANOLA<ESPANOLA<<CARMEN<<<<<


Hash numero documento:  4
Hash fecha nacimiento:  4
Hash control final:  1


## VERIFICACIÓN DE LA CONSISTENCIA INTERNA

Primero se definiran funciones para la extracción de datos de la parte delantera y una que las englobe.

In [6]:
def leerDNI(path):
    text_front = extract_text(model, path, angulo=2, algoritmo=Image.Resampling.BICUBIC, idCorte=2)
    
    dni = text_front.pages[0].blocks[0].lines[0].words[0].value
    
    try:
        int(dni) # Si el DNI detecta mal la letra (Q por 0 por ejemplo) no error y 2º intento
        text_front2 = extract_text(model, path, angulo=0.5, algoritmo=Image.Resampling.BICUBIC, idCorte=10)
        dni = text_front2.pages[0].blocks[0].lines[0].words[0].value
    except ValueError:
        return dni  
    
    try:
        int(dni) # Si el DNI detecta mal la letra otra vez devolver error
        return ""
    except ValueError:
        return dni

def leerApellidos(path):
    try:
        text_front = extract_text(model, path, angulo=2, algoritmo=Image.Resampling.BICUBIC, idCorte=3)

        apellidos = {
                "apellido1": text_front.pages[0].blocks[0].lines[1].words[0].value,
                "apellido2": text_front.pages[0].blocks[0].lines[2].words[0].value,
            }

        if apellidos["apellido1"] == "":
            return ""
        
        return apellidos
    
    except:
        return ""
    
def leerNombre(path):
    try:
        text_front = extract_text(model, path, angulo=2, algoritmo=Image.Resampling.BICUBIC, idCorte=4)
        nombre = {
                "nombre": text_front.pages[0].blocks[0].lines[1].words[0].value,
            }
        
        if nombre["nombre"] == "":
            return ""
        
        return nombre
    except:
        return ""

def leerNombreCompleto(path):
    try:
        nombre = leerNombre(path)
        apellidos = leerApellidos(path)

        nombre_completo = {
            "nombre": nombre["nombre"],
            "apellido1": apellidos["apellido1"],
            "apellido2": apellidos["apellido2"]
        }

        if nombre_completo["nombre"] == "" or nombre_completo["apellido1"] == "":
            return ""
        
        return nombre_completo

    except:
        return ""

def leerSexo(path):
    try:
        text_front = extract_text(model, path, angulo=2, algoritmo=Image.Resampling.BICUBIC, idCorte=5)
        sexo = ""
        for page in text_front.pages:
            for block in page.blocks:
                for line in block.lines:
                    for word in line.words:
                        if word.value in ["M", "F"]:
                            sexo = word.value
                            break
                                        
        sexoLista = {
                "sexo": sexo
            }
        
        if sexoLista["sexo"] == "":
            return ""
        
        return sexoLista

    except:
        return ""

def leerNacionalidad(path):
    text_front = extract_text(model, path, angulo=2, algoritmo=Image.Resampling.BICUBIC, idCorte=6)
    nacionalidad = {
            "nacionalidad": text_front.pages[0].blocks[0].lines[1].words[0].value,
        }
    
    if nacionalidad["nacionalidad"] == "":
        return ""
    
    return nacionalidad

def leerFechaNac(path):
    try:
        text_front = extract_text(model, path, angulo=2, algoritmo=Image.Resampling.BICUBIC, idCorte=7)
        fechaNac = {
                "dia": text_front.pages[0].blocks[0].lines[1].words[0].value,
                "mes": text_front.pages[0].blocks[0].lines[2].words[0].value,
                "anyo": text_front.pages[0].blocks[0].lines[3].words[0].value
            }
        
        if fechaNac["dia"] == "" or fechaNac["mes"] == "" or fechaNac["anyo"] == "":
            return ""

        fecha_final = date(int(fechaNac["anyo"]), int(fechaNac["mes"]), int(fechaNac["dia"]))
        return fecha_final
    
    except:
        return ""

def leerFechaEmisionYSoporte(path):
    try:
        text_front = extract_text(model, path, angulo=2, algoritmo=Image.Resampling.BICUBIC, idCorte=8)
        datos = {
                "dia": text_front.pages[0].blocks[0].lines[1].words[0].value,
                "mes": text_front.pages[0].blocks[0].lines[2].words[0].value,
                "anyo": text_front.pages[0].blocks[0].lines[3].words[0].value,
                "soporte": text_front.pages[0].blocks[0].lines[5].words[0].value
            }
        
        if datos["dia"] == "" or datos["mes"] == "" or datos["anyo"] == "" or datos["soporte"] == "":
            return {
                "fecha_emisión": "",
                "num_soporte": ""
            }

        datosDicc = {
            "fecha_emisión": (date(int(datos["anyo"]), int(datos["mes"]), int(datos["dia"]))),
            "num_soporte": datos["soporte"]
        }
        
        return datosDicc
    
    except:
        return {
            "fecha_emisión": "",
            "num_soporte": ""
        }

def leerFechaValidez(path):
    try:
        text_front = extract_text(model, path, angulo=2, algoritmo=Image.Resampling.BICUBIC, idCorte=9)
        fechaVali = {
                "dia": text_front.pages[0].blocks[0].lines[1].words[0].value,
                "mes": text_front.pages[0].blocks[0].lines[2].words[0].value,
                "anyo": text_front.pages[0].blocks[0].lines[3].words[0].value
            }
        
        if fechaVali["dia"] == "" or fechaVali["mes"] == "" or fechaVali["anyo"] == "":
            return ""
        
        fecha_final = date(int(fechaVali["anyo"]), int(fechaVali["mes"]), int(fechaVali["dia"]))
        return fecha_final
    
    except:
        return ""

def leerDatos(path):
    datosCompletos = {
        "dni": leerDNI(path),
        "nombre_completo": leerNombreCompleto(path),
        "sexo": leerSexo(path)["sexo"],
        "nacionalidad": leerNacionalidad(path)["nacionalidad"],
        "fecha_nacimiento": leerFechaNac(path),
        "fecha_emision": leerFechaEmisionYSoporte(path)["fecha_emisión"],
        "fecha_validez": leerFechaValidez(path),
        "num_soporte": leerFechaEmisionYSoporte(path)["num_soporte"]
    }

    for key, value in datosCompletos.items():
        if value == "":
            print(f"Error al leer {key}. Por favor, repita la imagen.")
            return ""

    return datosCompletos

In [8]:
path = "./dni_fotos/front.jpg"
nombre_completo = leerDatos(path)

if nombre_completo != "":
    for key, value in nombre_completo.items():
        print(f"{key}: {value}")


dni: 99999999R
nombre_completo: {'nombre': 'CARMEN', 'apellido1': 'ESPANOLA', 'apellido2': 'ESPANOLA'}
sexo: F
nacionalidad: ESP
fecha_nacimiento: 1980-01-01
fecha_emision: 2021-06-02
fecha_validez: 2031-06-02
num_soporte: CAA000000


Y ahora se comparará con los datos de la parte trasera.

In [9]:
def leerDatosAtras(MRZ):
    nombreEntero = MRZ["linea3"].split("<<")
    apellidos = nombreEntero[0].split("<")
    nombre = nombreEntero[1]

    if len(apellidos) > 1:
        nombreCompleto = {
            "nombre": nombre,
            "apellido1": apellidos[0],
            "apellido2": apellidos[1]
        }
    else:
        nombreCompleto = {
            "nombre": nombre,
            "apellido1": apellidos[0],
            "apellido2": ""
        }
    
    datosAtras = {
        "dni": MRZ["linea1"][15:24],
        "nombre_completo": nombreCompleto,
        "sexo": MRZ["linea2"][7:8],
        "nacionalidad": MRZ["linea2"][15:18],
        "fecha_nacimiento": MRZ["linea2"][0:6],
        "fecha_validez": MRZ["linea2"][8:14],
        "num_soporte": MRZ["linea1"][5:14]
    }

    return datosAtras

def compararDatos(datosAtras, datosDelante):
    comparacion = {
        "dni": datosAtras["dni"] == datosDelante["dni"],
        "nombre_completo": datosAtras["nombre_completo"] == datosDelante["nombre_completo"],
        "sexo": datosAtras["sexo"] == datosDelante["sexo"],
        "nacionalidad": datosAtras["nacionalidad"] == datosDelante["nacionalidad"],
        "fecha_nacimiento": datosAtras["fecha_nacimiento"] == datosDelante["fecha_nacimiento"].strftime("%y%m%d"),
        "fecha_validez": datosAtras["fecha_validez"] == datosDelante["fecha_validez"].strftime("%y%m%d"),
        "num_soporte": datosAtras["num_soporte"] == datosDelante["num_soporte"]
    }

    return comparacion
 

In [11]:
atras = leerDatosAtras(leerMRZ("./dni_fotos/vback.jpg"))
delante = leerDatos("./dni_fotos/vfront.jpg")

comp = compararDatos(atras, delante)

for key, value in comp.items():
    print(f"{key}: {value}")

dni: True
nombre_completo: True
sexo: True
nacionalidad: True
fecha_nacimiento: True
fecha_validez: True
num_soporte: True


## Comprobaciones Extra
- 

## Exportación a JSON

## 